In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from os import path
try:
    import urllib
except ImportError:
    from urlparse import urlparse
import pandas
# import sys
# !{sys.executable} -m pip install lxml
import lxml
import html5lib

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:10.0.1) Gecko/20100101 Firefox/10.0.1'}

r = requests.get('https://www.statista.com/statistics/227249/greatest-gap-between-rich-and-poor-by-us-state/', headers=headers)

test = pandas.read_html(r.text)

root = BeautifulSoup(r.text, 'html.parser')
table = root.find("table").prettify()

panda_list = pandas.read_html(str(table), converters={"tbody": str})
panda_table = pandas.DataFrame(panda_list[0])
panda_table.columns = ['State', 'Gini Coefficient']